<div align="center">
 <img src="https://raw.githubusercontent.com/matheusmota/dataviz2018/master/resources/images/logo_facens_pos.png" width="150px">
 <h1> Dataviz - Data Science Specialization Program - FACENS</h1>
</div>
<br><br>
# Exercício 1 - Primeiro contato com o Kaggle
(valendo nota)

* **Data de entrega:** até o final da aula 
* **Professor:**  Matheus Mota
* **Aluno:** Murilo Spinoza de Arruda
* **RA:** 203144

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt

## Questão 1
**Enunciado:**  Este notebook está associado ao *Kaggle Dataset* chamado "Exercício 1". Este *Kaggle Dataset* possui dois arquivos em formato CSV (anv.csv e BR_eleitorado_2016_municipio ). Escolha um dos datasets disponíveis e já conhecidos, a seu critério. Uma vez definido o csv, escolha no mínimo 7 e no máximo 12 variáveis (colunas) que você avalia como sendo relevantes. 

### Prévia do dataset anv
**Colunas e exemplo de registro**

In [ ]:
dfAnv = pd.read_csv('../input/dataviz-facens-20182-aula-1-exerccio-2/anv.csv', delimiter=',')
dfAnv.info()

In [ ]:
dfAnv.head(1)

### Prévia do dataset eleitorado
**Colunas e exemplo de registro**

In [ ]:
dfEleitorado = pd.read_csv('../input/dataviz-facens-20182-aula-1-exerccio-2/BR_eleitorado_2016_municipio.csv', delimiter=',')
dfEleitorado.info()

In [ ]:
dfEleitorado.head(1)

**Verificando a importancia da coluna de gênero não informado**

In [ ]:
dfPorcentGenNaoInformado = round((dfEleitorado['gen_nao_informado'] / dfEleitorado['total_eleitores']) * 100,4) # .sort_values('gen_nao_informado', ascending=False)
dfPorcentGenNaoInformado.sort_values(ascending=False).head(13)

**Entendendo a repetição de nomes de municípios**

In [ ]:
nomesRepetidos = dfEleitorado['nome_municipio'].value_counts()
print('Municípios que tem nome repetido: ', len(nomesRepetidos[nomesRepetidos > 1]))

nomesRepetidos = (dfEleitorado['nome_municipio'] + ' (' + dfEleitorado['uf'] + ')').value_counts()
print('Municípios com o mesmo nome em um mesmo estado: ', len(nomesRepetidos[nomesRepetidos > 1]))

### O dataset escolhido foi do eleitorado

As primeiras variaveis selecionadas foram: 
1. uf
2. total_eleitores
3. gen_feminino
4. gen_masculino

Algumas que não serão utilizadas são:
* **cod_municipio_tse**, pois os valores não tem significado, diferente do nome da cidade, por ex;
* **gen_nao_informado**, pois representa uma parcela muito pequena de eleitores. Apenas 12 cidades têm entre 1 e 5% de eleitores que não informaram o gênero;

**nome_municipio** tem o problema de que alguns repetem entre os estados. Para solucionar isso, será criada uma nova coluna no seguinte formato: CIDADE (UF).

Essa abordagem facilitará a visualização e entendimento de gráficos que apresentarem o nome dos municípios, por dar o contexto do estado correspondente.

5. municipio

In [ ]:
dfEleitorado.insert(3, 'municipio', dfEleitorado['nome_municipio'] + ' (' + dfEleitorado['uf'] + ')')
dfEleitorado.head(1)

Para um melhor aproveitamento das informações e respeitar o número de variáveis proposto, serão criadas novas colunas para as faixas etarias de jovens e idosos que têm o voto facultativo.
6. f_inf_18
7. f_sup_69
8. facultativos

E, para contemplar o maior número de eleitores na análise, algumas faixas etárias serão agrupadas

9. f_18_24 (f_18_20 + f_21_24)
10. f_25_44 (f_25_34 + f_35_44)
11. f_45_69 (f_45_59 + f_60_69)

In [ ]:
dfEleitorado['f_inf_18'] = dfEleitorado['f_16'] + dfEleitorado['f_17']
dfEleitorado['f_sup_69'] = dfEleitorado['f_70_79'] + dfEleitorado['f_sup_79']
dfEleitorado['facultativos'] = dfEleitorado['f_inf_18'] + dfEleitorado['f_sup_69']
dfEleitorado['f_18_24'] = dfEleitorado['f_18_20'] + dfEleitorado['f_21_24']
dfEleitorado['f_25_44'] = dfEleitorado['f_25_34'] + dfEleitorado['f_35_44']
dfEleitorado['f_45_69'] = dfEleitorado['f_45_59'] + dfEleitorado['f_60_69']
dfEleitorado.head(1)

## DataFrame com as variáveis escolhidas

In [ ]:
variaveis = ['uf','municipio','total_eleitores','gen_feminino','gen_masculino','f_inf_18','f_18_24','f_25_44','f_45_69','f_sup_69','facultativos']
df = dfEleitorado[variaveis]
df.head(5)

Para cada uma das suas variáveis escolhidas, forneça:


### Questão 1 - Item A - Classificação das variáveis

Classifique todas as variáveis escolhidas, e construa um dataframe com sua resposta.
Exemplo:

In [ ]:
d = 'Quantitativa Discreta'
n = 'Qualitativa Nominal'
resposta = {
    'Variavel': variaveis,
    'Classificação': [n, n, d,d,d,d,d,d,d,d,d]
}
resposta = pd.DataFrame(resposta)
resposta

### Questão 1 - Item B - Tabela de frequência

Construa uma tabela de frequência para cada uma das **variáveis qualitativas** que você escolheu (caso não tenha escolhido nenhuma, deixe esta questão em branco). Uma dica: a função *value_counts()* do Pandas pode ser muito útil. =)


In [ ]:
freqUf = pd.DataFrame(df['uf'].value_counts().sort_values(ascending=False))
freqUf

In [ ]:
freqMunicipio = pd.DataFrame(df['municipio'].value_counts().sort_values(ascending=False))
freqMunicipio

### Questão 1 - Item C - Representação Gráfica
Para cada uma das variáveis, produza um ou mais gráficos, usando matplotlib, que descreva seu comportamento / caracteristica. Lembre-se que estes gráficos precisam ser compatíveis com a classificação da variável. 

In [ ]:
def scatterPlotForLatLong(df, title, s, c=None):
    if c is None:
        c = s
    return df.plot(
        kind="scatter", x="longitude", y="latitude", alpha=0.4, legend=False,
        title=title,
        s=s/3500, #label="eleitorado",
        c=c, figsize=(10,7),
        cmap=plt.get_cmap("jet"), colorbar=True
    )

## !!! Para a apresentação da variável de estado e cidade, fiz o processamento de uma base externa de estados e municípios para obter a informação da LATITUDE E LONGITUDE de cada munícipio do nosso dataset

O dataset está disponivel no seguinte link: https://github.com/kelvins/Municipios-Brasileiros/tree/main/csv

Peguei as latitudes e longitudes dos ESTADOS e inseri no bloco a seguir.

In [ ]:
totalEleitorado = df.groupby(['uf'])['total_eleitores'].agg('sum')

dfUfs = pd.DataFrame({
    'longitude': [-70.55,-36.82,-65.10,-51.77,-41.71,-39.53,-40.34,-49.86,-45.44,-44.38,-54.54,-55.42,-52.48,-36.72,-37.86,-42.28,-51.55,-42.66,-36.59,-63.34,-61.33,-53.50,-50.95,-37.45,-48.79,-48.26],
    'latitude':  [-8.77,-9.62,-3.47,1.41,-13.29,-5.20,-19.19,-15.98,-5.42,-18.10,-20.51,-12.64,-3.79,-7.28,-8.38,-6.60,-24.89,-22.25,-5.81,-10.83,1.99,-30.17,-27.45,-10.57,-22.19,-9.46],
    'uf': list(totalEleitorado.index),
    'eleitorado': totalEleitorado.values
})

COM A BASE DE MUNICÍPIOS, gerei um dicionario para associar o nome da cidade com sua respectiva latitude e longitude. Feito isso, salvei no json associado ao notebook **(cidadeLatlong.json)**.

Na célula abaixo, incluo a latitude e longitude no dataframe

In [ ]:
df = df.assign(longitude=np.nan)
df = df.assign(latitude=np.nan)
with open('../input/cidades-latitude-longitude/cidadeLatlong.json', 'r') as cidadesLatlong:
    json_object = json.loads(cidadesLatlong.read())
    for index, value in df.iterrows():
        latlong = json_object[value['municipio']]
        df.loc[index, 'longitude'] = latlong['longitude']
        df.loc[index, 'latitude'] = latlong['latitude']
df.head()

# UF X Eleitorado

In [ ]:
ax = scatterPlotForLatLong(
    dfUfs,
    'Dispersão do eleitorado nacional em cada estado (%)',
    dfUfs["eleitorado"],
    dfUfs["eleitorado"]/dfUfs["eleitorado"].sum()*100
)

for index, value in dfUfs.iterrows():
    annotate = ax.annotate(value['uf'], (value['longitude'] -0.62, value['latitude']-0.35))


# Cidades X Eleitorado

In [ ]:
ax = scatterPlotForLatLong(
    df,
    'Dispersão de eleitores nas cidades',
    df["total_eleitores"]
)

ax = scatterPlotForLatLong(
    df,
    'Eleitores facultativos com menos de 18 anos nas cidades',
    df["f_inf_18"]
)

ax = scatterPlotForLatLong(
    df,
    'Eleitores facultativos com mais de 69 anos nas cidades',
    df["f_sup_69"]
)


# Eleitores por gênero

In [ ]:
totalEleitores = df['total_eleitores'].sum()
plot = pd.DataFrame(
    {'Total': [df['gen_masculino'].sum() , df['gen_feminino'].sum()]},
    index=['Masculino', 'Feminino']
).plot.pie(y='Total',autopct='%1.1f%%', title='Total de eleitores por gênero', startangle=20)

# Estados X Eleitores por gênero

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))
dfGenero = df.groupby('uf').agg('sum')[['gen_feminino',"gen_masculino"]].sort_values('uf', ascending=False)
dfGenero['percent_gen_masculino'] = round(dfGenero['gen_masculino'] / (dfGenero['gen_feminino'] + dfGenero['gen_masculino']) * 100, 1)
dfGenero['percent_gen_feminino'] = round(dfGenero['gen_feminino'] / (dfGenero['gen_feminino'] + dfGenero['gen_masculino']) * 100, 1)
dfGenero = dfGenero.drop(['gen_feminino', 'gen_masculino'], axis=1)
dfGenero.plot(kind='barh', ax=ax, grid=True)
plt.legend(['Masculino', 'Feminino'])
plt.title("Porcentagem de eleitores de cada gênero nos estados")
label = plt.ylabel("Estados")

# Eleitores X Faixas etárias

In [ ]:
totalEleitores = df['total_eleitores'].sum()
faixasEtarias = {
    '< 18':   (df['f_inf_18'].sum() / totalEleitores) * 100,
    '18 a 24':(df['f_18_24'].sum()  / totalEleitores) * 100,
    '25 a 44':(df['f_25_44'].sum()  / totalEleitores) * 100,
    '45 a 69':(df['f_45_69'].sum()  / totalEleitores) * 100,
    '> 69':   (df['f_sup_69'].sum() / totalEleitores) * 100,
}

names = list(faixasEtarias.keys())
values = list(faixasEtarias.values())

fig, ax = plt.subplots(figsize=(8,5))
ax.bar(names, values)

for i, v in enumerate(values):
    ax.text(i - 0.2 ,v + 0.5, "{0:.2f}%".format(v))

title = plt.title("Porcentagem do total de eleitores por faixa etária")

# Estados X Eleitores por faixas etárias

In [ ]:
faixasEtarias = ['f_inf_18', 'f_18_24', 'f_25_44', 'f_45_69', 'f_sup_69']
dfUfIdades = df.groupby('uf').agg('sum')[faixasEtarias + ['total_eleitores']].sort_values('uf', ascending=False)
for faixaEtaria in faixasEtarias:
    dfUfIdades[faixaEtaria] = dfUfIdades[faixaEtaria] / dfUfIdades['total_eleitores'] * 100
dfUfIdades.drop('total_eleitores', axis=1, inplace=True)
plot = dfUfIdades.plot(kind='bar', grid=True, figsize=(22,8), stacked=True, cmap=plt.get_cmap('Set3'))

# Estados X Votos facultativos

In [ ]:
dfFacultativos = df.groupby('uf').agg('sum')[['facultativos', 'total_eleitores']]
dfFacultativos = dfFacultativos.sort_values('uf', ascending=False)
dfFacultativos['facultativos'] = dfFacultativos['facultativos'] / dfFacultativos['total_eleitores'] * 100
dfFacultativos.drop('total_eleitores', axis=1, inplace=True)
plot = dfFacultativos.plot(
    kind='bar', grid=True, figsize=(22,8), stacked=True, cmap=plt.get_cmap('Set3'),
    title='Percentual de votos facultativos por estado'
)